# TP &mdash; Restricted Boltzmann Machines

**Auteurs:**

* Javier Andres Boada Martinez
* Kevin Alexandro Sanchez Diaz
* Yevhenii Sielskyi

## Chargement de données

In [ ]:
import scipy.io
import numpy as np
from matplotlib import pyplot as plt

alphadigits = scipy.io.loadmat('data/binaryalphadigs.mat')

## Analyse Exploratoire des Données 

In [ ]:
# 10 digits and 26 handwritten letters (A-Z),
# 39 examples of each class: 
print(f"There are {alphadigits['dat'].shape[0]} different character classes, "
      f"{alphadigits['dat'].shape[1]} examples each.")

In [ ]:
# The dimensions of each image:
print(f"The size of each binary image is: {alphadigits['dat'][0, 0].shape}")

In [ ]:
# Number of random classes to output:
nb_classes_to_show = 5
# Examples per class to show:
nb_samples_per_class = 3

# Images to show in one row:
nb_imgs_per_row = 3
# Number of rows:
nb_rows = nb_samples_per_class // nb_imgs_per_row + 1 \
            if nb_samples_per_class % nb_imgs_per_row \
            else nb_samples_per_class // nb_imgs_per_row

# Randomly choose some classes (their indices):
class_indices = np.sort(np.random.choice(
    range(alphadigits['dat'].shape[0]),
    size=nb_classes_to_show
))

# Output some images:
print("Training data examples:")
for class_index in class_indices:
    print(f"\nClass {class_index}:")

    # Choose random samples:
    example_indices = np.random.choice(
        range(alphadigits['dat'].shape[1]),
        size=nb_samples_per_class
    )

    # Output chosen class images:
    plt.figure(figsize=(5 * nb_imgs_per_row, 6 * nb_rows))
    for i, image in enumerate(alphadigits['dat'][class_index, example_indices], 1):
        plt.subplot(nb_rows, nb_imgs_per_row, i)
        plt.imshow(image)
    plt.show()

## Échantillonnage des données

In [ ]:
def read_alpha_digits(data, indices):
    dataset = []
    
    for idx in indices:
        dataset += [image.reshape(-1) for image in data[idx]]

    return np.array(dataset)

In [ ]:
# Character classes to learn:
train_classes = [10, 11]

In [ ]:
dataset = read_alpha_digits(alphadigits['dat'], train_classes)

In [ ]:
dataset.shape

## Entraînement

In [ ]:
from rbm import RBM

In [ ]:
# Hyperparameters:
HIDDEN_UNITS = 200
LEARNING_RATE = 1E-2

EPOCHS = 500
BATCH_SIZE = 10

In [ ]:
rbm = RBM(dataset.shape[-1], HIDDEN_UNITS)

In [ ]:
rbm.train(dataset, lr=LEARNING_RATE, epochs=EPOCHS, batch_size=BATCH_SIZE)

## Génération des images

In [ ]:
# Image generation parameters:
ITER_GIBBS = 100
NB_IMAGES = 5

In [ ]:
test_images = rbm.generate_images(iter_gibbs=ITER_GIBBS, nb_images=NB_IMAGES)

In [ ]:
# Number of rows:
nb_rows = NB_IMAGES // nb_imgs_per_row + 1 \
            if NB_IMAGES % nb_imgs_per_row \
            else NB_IMAGES // nb_imgs_per_row

# Output generated images:
print("Generated images:")
plt.figure(figsize=(5 * nb_imgs_per_row, 6 * nb_rows))
for i, image in enumerate(test_images, 1):
    plt.subplot(nb_rows, nb_imgs_per_row, i)
    plt.imshow(image)
plt.show()

## Expériences

## MNIST

In [ ]:
# load MNIST data - handwritten digits:
mnist = scipy.io.loadmat('data/mnist_all.mat')

# MNIST image size:
mnist_img_size = (28, 28)

In [ ]:
# Merge training and test data for each class:
for class_index in range(10):
    mnist['train' + str(class_index)] = np.concatenate(
        (mnist['train' + str(class_index)],
        mnist['test' + str(class_index)]), axis=0
    )

In [ ]:
# Character classes to learn:
train_classes = [5]

# Create a MNIST training dataset:
mnist_dataset = np.concatenate(
    [mnist['train' + str(class_index)] for class_index in train_classes],
    axis=0
)

# Binarise data:
mnist_dataset = np.where(mnist_dataset > 0, 1, 0)

In [ ]:
# Create another RBM:
mnist_rbm = RBM(mnist_dataset.shape[-1], HIDDEN_UNITS)

# Train RBM on the MNIST data:
mnist_rbm.train(
    mnist_dataset, lr=LEARNING_RATE,
    epochs=EPOCHS, batch_size=BATCH_SIZE * 10
)

In [ ]:
test_images = mnist_rbm.generate_images(
    iter_gibbs=ITER_GIBBS * 5,
    nb_images=NB_IMAGES,
    img_size=mnist_img_size    
)

In [ ]:
# Number of rows:
nb_rows = NB_IMAGES // nb_imgs_per_row + 1 \
            if NB_IMAGES % nb_imgs_per_row \
            else NB_IMAGES // nb_imgs_per_row

# Output generated images:
print("Generated images:")
plt.figure(figsize=(5 * nb_imgs_per_row, 5 * nb_rows))
for i, image in enumerate(test_images, 1):
    plt.subplot(nb_rows, nb_imgs_per_row, i)
    plt.imshow(image)
plt.show()

## USPS

In [ ]:
# load USPS data - handwritten digits:
usps = scipy.io.loadmat('data/usps_all.mat')

# USPS image size:
usps_img_size = (16, 16)

In [ ]:
# Character classes to learn:
train_classes = [5]

usps_dataset = np.concatenate(
    [usps['data'][:, :, class_index].T for class_index in train_classes],
    axis=0
)

# Binarise data:
usps_dataset = np.where(usps_dataset > 0, 1, 0)

In [ ]:
# Hyperparameters:
HIDDEN_UNITS = 300
LEARNING_RATE = 1E-3

EPOCHS = 2000
BATCH_SIZE = 100

In [ ]:
# Create another RBM:
usps_rbm = RBM(usps_dataset.shape[-1], HIDDEN_UNITS)

# Train RBM on the USPS data:
usps_rbm.train(
    usps_dataset, lr=LEARNING_RATE,
    epochs=EPOCHS, batch_size=BATCH_SIZE
)

In [ ]:
test_images = usps_rbm.generate_images(
    iter_gibbs=ITER_GIBBS * 5,
    nb_images=NB_IMAGES,
    img_size=usps_img_size
)

In [ ]:
# Number of rows:
nb_rows = NB_IMAGES // nb_imgs_per_row + 1 \
            if NB_IMAGES % nb_imgs_per_row \
            else NB_IMAGES // nb_imgs_per_row

# Output generated images:
print("Generated images:")
plt.figure(figsize=(5 * nb_imgs_per_row, 5 * nb_rows))
for i, image in enumerate(test_images, 1):
    plt.subplot(nb_rows, nb_imgs_per_row, i)
    plt.imshow(image.T)
plt.show()